In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install 'h5py==2.10.0' --force-reinstall

     |████████████████████████████████| 2.9MB 4.0MB/s 
     |████████████████████████████████| 15.7MB 329kB/s 
ERROR: lucid 0.3.10 requires umap-learn, which is not installed.
ERROR: tensorflow 1.15.2 has requirement gast==0.2.2, but you'll have gast 0.4.0 which is incompatible.
ERROR: lucid 0.3.10 has requirement numpy<=1.19, but you'll have numpy 1.21.0 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.15.0, but you'll have six 1.16.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: six 1.15.0
    Uninstalli

In [ ]:
# import tensorflow as tf
# tf.__version__

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

In [ ]:
import keras

Using TensorFlow backend.


In [ ]:
keras.__version__

'2.3.1'

In [ ]:
import numpy as np
from sklearn import preprocessing
import pandas as pd
from sklearn.preprocessing import normalize
from keras.layers import Input, Conv2D,  Dense, Flatten,MaxPooling2D, concatenate
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from itertools import permutations, combinations, combinations_with_replacement
from keras.layers import LSTM, Permute,Reshape
import keras
from sklearn.model_selection import train_test_split
import collections
import random
import tensorflow as tf
import gc
from tqdm import tqdm
import glob

from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.keras import backend
from tensorflow.python.keras.engine.base_layer import Layer
# from tensorflow.python.keras.utils import control_flow_util
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import nn
from tensorflow.python.util.tf_export import keras_export

Using TensorFlow backend.


In [ ]:
# =============================================================================
# MobilityAI
# =============================================================================
#MobilityAI data
x = np.array([1,2,5,10,20])
stl = np.array([0.5123,0.6667,0.7736,0.8423,0.9158])
msens = np.array([0.8879,0.9237,0.9460,0.9544,0.9635])
bmtl =np.array([0.8471,0.9388,0.9324,0.9391,0.9518])
tmtl = np.array([0.9227,0.9298,0.9508,0.9541,0.9640])
# example error bar values that vary with x-position
error1 = np.array([0.1087,0.0889,0.0905,0.0512,0.0325])/np.sqrt(4)
error2 = np.array([0.0505,0.0398,0.0315,0.0267,0.0270])/np.sqrt(4)
error3 = np.array([0.0521,0.0458,0.0411,0.0475,0.0367])/np.sqrt(4)
error4 = np.array([0.0605,0.0573,0.0238,0.0311,0.0251])/np.sqrt(4)
men_means, men_std = (20, 35, 30, 35, 27), (2, 3, 4, 1, 2)
women_means, women_std = (25, 32, 34, 20, 25), (3, 5, 2, 3, 3)
ind = np.arange(len(men_means))  # the x locations for the groups
width = 0.2  # the width of the bars
fig, ax = plt.subplots(figsize=(8, 4))
rects1 = ax.bar(ind - width*2, stl, width, yerr=error1,capsize=5,
                label='STL')
rects2 = ax.bar(ind - width, msens, width, yerr=error2,capsize=5,
                label='MetaSense')
rects3 = ax.bar(ind , bmtl, width, yerr=error3,capsize=5,
                label='BMTL')
rects4 = ax.bar(ind + width, tmtl, width, yerr=error4,capsize=5,
                label='TMTL')
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Test Accuracy')
ax.set_xlabel('Number of Data Windows')
ax.set_title('MobilityAI')
ax.set_xticks(ind)
ax.set_xticklabels(('1', '2', '5', '10', '20'))
ax.set_ylim([0.4,1.0])
ax.legend()
plt.tight_layout()
plt.show()

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# from tensorflow.python.util import deprecation
# deprecation._PRINT_DEPRECATION_WARNINGS = False

In [ ]:
# tf.compat.v1.experimental.output_all_intermediates(True)
# tf.compat.v1.enable_resource_variables()

In [ ]:
tf.__version__

'1.15.2'

In [ ]:
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
ALPHA = 2
#number of activities
NUM_CLASSES = 4
 
# Hardcoded length of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_LENGTH = 60
 
# Hardcoded step of the sliding window mechanism employed to segment the data
# SLIDING_WINDOW_STEP = 160
 
# Batch Size
BATCH_SIZE = 100
 
# Number filters convolutional layers
NUM_FILTERS = 64
 
# Size filters convolutional layers
FILTER_SIZE = 5
 
# Number of unit in the long short-term recurrent layers
NUM_UNITS_LSTM = 128
 
NUM_CHANNEL = 9

In [ ]:
#flatten data into matrix(28x28-->784),for sensor data no need to do this 
# =============================================================================
# Load and preprocess training and testing data
# =============================================================================
 
# to load the windowed data
def load_tensor(path, sensor):
    # Read the array from disk
    #new_data = np.loadtxt('./ActiveData/Sub'+str(name)+'_data.txt')
    new_data = np.loadtxt(path)
    
    #changing NUM_CHANNEL according to sensor combination type
    combination = ['WT', 'AW']
    single = ['A', 'W', 'T']
    if sensor in combination:
        NUM_CHANNEL = 6
    elif sensor in single:
        NUM_CHANNEL = 3
    else:
        NUM_CHANNEL = 9
 
    # Note that this returned a 2D array!
    # print(new_data.shape)
    
    # However, going back to 3D is easy if we know the 
    # original shape of the array
    if sensor == "All":
        pass
    elif sensor == "WT":
        new_data = new_data[:,:6]
    elif sensor == "AW":
        new_data = new_data[:,3:]
    elif sensor == 'A':
        new_data = new_data[:,6:]
    elif sensor == 'W':
        new_data = new_data[:,3:6]
    elif sensor == 'T':
        new_data = new_data[:,:3]
 
    new_data = new_data.reshape((-1, SLIDING_WINDOW_LENGTH, NUM_CHANNEL))
 
    return new_data, NUM_CHANNEL
 
def load_by_trial(trial_id, arg, sensor): 
    #arg can be either 'train' or 'test'
    path = '/content/drive/MyDrive/MITACS_Nithin/juravinski/data/trial'+str(trial_id)+'/*_'+arg+'_data.txt'
    x_list = glob.glob(path)
    # path = '/content/drive/MyDrive/MITACS/juravinski/data/trial'+str(trial_id)+'/*_train_label.txt'
    # y_list = glob.glob(path)
    y_list = [i[0:-8]+'label.txt' for i in x_list]
    X,Y = [],[]
    for i in tqdm(range(len(x_list)-19)):
        x, channels = load_tensor(x_list[i], sensor)
        y = np.loadtxt(y_list[i])
        X.append(x)
        Y.append(y)
    # x_ = convert_list(X)
    # y_ = convert_y(Y)
    # idx = np.where(y_==4)
    # y_[idx] = 3 
    return X, Y, channels

In [ ]:
# =============================================================================
# load data and train
# in the order of ['thigh','wrist','ankle']
# =============================================================================
sensor = 'All'
x1,y1,channels = load_by_trial(1, 'train', sensor)
x2,y2,channels = load_by_trial(2, 'train', sensor)
X = []
Y = []
 
for i in range(10):
    x = np.concatenate((x1[i], x2[i]), axis=0)I hope you received the MILA link previously; if not, I may resend it.
    y1_ = y1[i]
    idx = np.where(y1_==4)
    y1_[idx] = 3
    y2_ = y2[i]
    idx = np.where(y2_==4)
    y2_[idx] = 3
    y = np.concatenate((y1_, y2_), axis=0)
    X.append(x)
    Y.append(y)
 
# print(X_train[0].shape)
# print(y_train[0].shape)

y_train,y_val, X_train, X_val = [],[],[],[]
for i in range(10):
    x_t, x_v, y_t, y_v = train_test_split(X[i],Y[i],stratify=Y[i],test_size=0.1,random_state=42)
    X_train.append(x_t)
    X_val.append(x_v)
    y_train.append(y_t)
    y_val.append(y_v) 
                 
    
import collections
 
# X_train = np.vstack((x1,x2))
# y_train = np.concatenate((y1,y2))
# print(collections.Counter(y_train))
 
# from imblearn.over_sampling import RandomOverSampler
# oversample = RandomOverSampler()
# X_train = X_train.reshape(-1,SLIDING_WINDOW_LENGTH*channels)
# X_train, y_train = oversample.fit_resample(X_train, y_train)
# print(collections.Counter(y_train))
# X_train = X_train.reshape(-1,1,SLIDING_WINDOW_LENGTH,channels)
 
 
labels = np.array([0,1,2,3])
lb = preprocessing.LabelBinarizer()
lb.fit(y=labels)
 
# y_onehot_train = lb.transform(y_train) 
NUM_CHANNEL = channels
 
# print(len(X_train))
 
#3. flatten the 60X12 part ---- why??
x_train_flat = []
x_val_flat = []
 
for i in range(10):
    x_train_flat.append(X_train[i].reshape(-1,540))
    x_val_flat.append(X_val[i].reshape(-1,540))
    #the fewest label is 16 and 161 respectively, so pick m=16 n=160
    print(collections.Counter(y_train[i].ravel()))
    print(collections.Counter(y_val[i].ravel()))

In [ ]:
# =============================================================================
# Replicate triplet nn method
# 1. preprocess data into triplets
#   This data generator has to be updated as the memory can't bear all these samples
# 2. construct triplet nn
# =============================================================================
def generate_triplet(x,y,ap_pairs,an_pairs):
    """
    To generate triplet from original dataset
    Arguments:
    ap_pairs -- how many random anchor-positive samples generate for a given class
    an_pairs -- how many random anchor-negative samples generate for a given class
                Does not required to be the same as ap_pairs, but for banlancing, should set the same?
    
    Returns:
    triplet_pairs -- 3d tensor; its shape[0]=NUM_CLASSES*ap_pairs*an_pairs
    """
    data_xy = tuple([x,y])
 
    triplet_pairs = []
    labels = []
   
    for data_class in sorted(set(data_xy[1])):
        same_class_idx = np.where((data_xy[1] == data_class))[0]
        diff_class_idx = np.where(data_xy[1] != data_class)[0]
        
        if same_class_idx.shape[0] == 1:
               same_ = [(same_class_idx[0],same_class_idx[0])]
        else:
            same_ = list(combinations(same_class_idx,2))
        diff_ = list(diff_class_idx)
        if len(same_) >= ap_pairs:
            A_P_pairs = random.sample(same_,k=ap_pairs) #Generating Anchor-Positive pairs
        else:
            A_P_pairs = same_
        if len(diff_) >= an_pairs:
            Neg_idx = random.sample(diff_,k=an_pairs)
        else:
            Neg_idx = diff_
        
        #Put data into triplets according to the indices in (A_P_pairs and Neg_idx)
        A_P_len = len(A_P_pairs)
        #print(A_P_len)
        for ap in A_P_pairs[:int(A_P_len)]:
            Anchor = data_xy[0][ap[0]]
            Positive = data_xy[0][ap[1]]
            for n in Neg_idx:
                Negative = data_xy[0][n]
                triplet_pairs.append([Anchor,Positive,Negative])
                labels.append(data_class)
                
    return np.array(triplet_pairs),np.array(labels)

In [ ]:
# =============================================================================
# Loss function
# =============================================================================
def triplet_loss(model, alpha = 0.4):
    """
    Implementation of the triplet loss function
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor data
            positive -- the encodings for the positive data (similar to anchor)
            negative -- the encodings for the negative data (different from anchor)
    Returns:
    loss -- real number, value of the loss
    """
    myy_pred = model.get_layer("merged_layer").output
    #print('triplet NN.shape = ',myy_pred)
    
    total_lenght = myy_pred.shape.as_list()[-1]
#     print('total_lenght=',  total_lenght)
#     total_lenght =12
    
    anchor = myy_pred[:,0:int(total_lenght*1/3)]
    positive = myy_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)]
    negative = myy_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]
 
    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)
 
    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)
 
    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
 
    return loss

In [ ]:
def multi_task_loss(model, alpha = 1):
    def _loss(y_true, y_pred):
        myloss = K.mean(K.binary_crossentropy(y_true, y_pred))
        miu = 0.8
        S0 = model.layers[5].get_weights()[0]
        S1 = model.layers[6].get_weights()[0]
        S2 = model.layers[7].get_weights()[0]
        S3 = model.layers[8].get_weights()[0]
        
        S4 = model.layers[9].get_weights()[0]
        S5 = model.layers[10].get_weights()[0]
        S6 = model.layers[11].get_weights()[0]
        S7 = model.layers[12].get_weights()[0]
        
        S8 = model.layers[13].get_weights()[0]
        S9 = model.layers[14].get_weights()[0]
        
        S = np.hstack((S0,S1,S2,S3,S4,S5,S6,S7,S8,S9))
#        myloss+= miu*np.linalg.norm(S[0].reshape(-1,5),ord=1)
        myloss+= miu*np.linalg.norm(S,ord=1)
       
        return myloss*alpha + triplet_loss(model)
    return _loss

In [ ]:
class BaseDenseAttention(keras.layers.Layer):
  def __init__(self, causal=False, dropout=0.0,
               **kwargs):
    super(BaseDenseAttention, self).__init__(**kwargs)
    self.causal = causal
    self.dropout = dropout
    self.supports_masking = True

  def _calculate_scores(self, query, key):
    return NotImplementedError

  def _apply_scores(self, scores, value, scores_mask=None, training=None):
    if scores_mask is not None:
      padding_mask = math_ops.logical_not(scores_mask)
      # Bias so padding positions do not contribute to attention distribution.
      # Note 65504. is the max float16 value.
      if scores.dtype is dtypes.float16:
        scores -= 65504. * math_ops.cast(padding_mask, dtype=scores.dtype)
      else:
        scores -= 1.e9 * math_ops.cast(padding_mask, dtype=scores.dtype)
    if training is None:
      training = backend.learning_phase()
    weights = nn.softmax(scores)

    def dropped_weights():
      return nn.dropout(weights, rate=self.dropout)

    # weights = control_flow_util.smart_cond(training, dropped_weights,
    #                                        lambda: array_ops.identity(weights))
    return math_ops.matmul(weights, value), weights

  # TODO(b/125916026): Consider exposing a __call__ method with named args.
  def call(self,
           inputs,
           mask=None,
           training=None,
           return_attention_scores=False):
    self._validate_call_args(inputs=inputs, mask=mask)
    q = inputs[0]
    v = inputs[1]
    k = inputs[2] if len(inputs) > 2 else v
    q_mask = mask[0] if mask else None
    v_mask = mask[1] if mask else None
    scores = self._calculate_scores(query=q, key=k)
    if v_mask is not None:
      # Mask of shape [batch_size, 1, Tv].
      v_mask = array_ops.expand_dims(v_mask, axis=-2)
    if self.causal:
      # Creates a lower triangular mask, so position i cannot attend to
      # positions j>i. This prevents the flow of information from the future
      # into the past.
      scores_shape = array_ops.shape(scores)
      # causal_mask_shape = [1, Tq, Tv].
      causal_mask_shape = array_ops.concat(
          [array_ops.ones_like(scores_shape[:-2]), scores_shape[-2:]],
          axis=0)
      causal_mask = _lower_triangular_mask(causal_mask_shape)
    else:
      causal_mask = None
    # scores_mask = _merge_masks(v_mask, causal_mask)
    result, attention_scores = self._apply_scores(
        scores=scores, value=v, training=training) #scores_mask=scores_mask
    if q_mask is not None:
      # Mask of shape [batch_size, Tq, 1].
      q_mask = array_ops.expand_dims(q_mask, axis=-1)
      result *= math_ops.cast(q_mask, dtype=result.dtype)
    if return_attention_scores:
      return result, attention_scores
    return result

  def compute_mask(self, inputs, mask=None):
    self._validate_call_args(inputs=inputs, mask=mask)
    if mask:
      q_mask = mask[0]
      if q_mask is None:
        return None
      return ops.convert_to_tensor_v2_with_dispatch(q_mask)
    return None

  def _validate_call_args(self, inputs, mask):
    """Validates arguments of the call method."""
    class_name = self.__class__.__name__
    if not isinstance(inputs, list):
      raise ValueError(
          '{} layer must be called on a list of inputs, namely [query, value] '
          'or [query, value, key].'.format(class_name))
    if len(inputs) < 2 or len(inputs) > 3:
      raise ValueError(
          '{} layer accepts inputs list of length 2 or 3, '
          'namely [query, value] or [query, value, key]. '
          'Given length: {}'.format(class_name, len(inputs)))
    if mask:
      if not isinstance(mask, list):
        raise ValueError(
            '{} layer mask must be a list, '
            'namely [query_mask, value_mask].'.format(class_name))
      if len(mask) < 2 or len(mask) > len(inputs):
        raise ValueError(
            '{} layer mask must be a list of length 2, namely [query_mask, '
            'value_mask]. Given length: {}'.format(class_name, len(mask)))

  def get_config(self):
    config = {
        'causal': self.causal,
        'dropout': self.dropout,
    }
    base_config = super(BaseDenseAttention, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))


# @keras_export('keras.layers.Attention')
class Attention(BaseDenseAttention):
  def __init__(self, use_scale=False, **kwargs):
    super(Attention, self).__init__(**kwargs)
    self.use_scale = use_scale

  def build(self, input_shape):
    """Creates scale variable if use_scale==True."""
    if self.use_scale:
      self.scale = self.add_weight(
          name='scale',
          shape=(),
          initializer=init_ops.ones_initializer(),
          dtype=self.dtype,
          trainable=True)
    else:
      self.scale = None
    super(Attention, self).build(input_shape)

  def _calculate_scores(self, query, key):
    scores = math_ops.matmul(query, key, transpose_b=True)
    if self.scale is not None:
      scores *= self.scale
    return scores

  def get_config(self):
    config = {'use_scale': self.use_scale}
    base_config = super(Attention, self).get_config()
    return dict(list(base_config.items()) + list(config.items()))

In [ ]:
# =============================================================================
# Construct the neural network
# =============================================================================
#The 'Net' part, will be replaced by 4CNN+2LSTM
def Conv_Att_LSTM(cpu=False, attention = "Luong", dropout = 0.2):  #specify type of attention as a param

    multi_input = Input(shape=(1, SLIDING_WINDOW_LENGTH, NUM_CHANNEL), name='multi_input')
    format = 'channels_first'

    if attention == "Luong":
        attention1 = Attention(dropout = dropout)
        attention2 = Attention(dropout = dropout)
    # elif attention == 'MultiHead':
    #     attention1 = MultiHeadAttention(num_heads=4)
    #     attention2 = MultiHeadAttention(num_heads=4)
    
    
    y = Conv2D(64, (5, 1), activation='relu', data_format=format)(multi_input)

    y = Conv2D(64, (5, 1), activation='relu', data_format=format)(y)

    y = Conv2D(64, (5, 1), activation='relu', data_format=format)(y)

    y = Conv2D(64, (5, 1), activation='relu', data_format=format)(y

    y = Permute((2, 1, 3))(y)

    y = Reshape((int(y.shape[1]), int(y.shape[2]) * int(y.shape[3])))(y) #(44,576)

    #adding attention layer, type of attention specified above

    y = attention1([y,y]) #attention layer - more like self attention but used along with conv and lstm
    
    y = LSTM(128,dropout=0.25,return_sequences=True)(y) #(44,128)

    y = attention2([y,y])

    y = LSTM(128)(y)

    # y = attention([y,y])

    return Model(inputs=multi_input, outputs=y) 

In [ ]:
# =============================================================================
# Construct the neural network
# =============================================================================
#The 'Net' part, will be replaced by 4CNN+2LSTM
def Conv_Att_LSTM(cpu=False, attention = "Luong", dropout = 0.2):  #specify type of attention as a param

    multi_input = Input(shape=(1, SLIDING_WINDOW_LENGTH, NUM_CHANNEL), name='multi_input')
    format = 'channels_first'

    if attention == "Luong":
        attention1 = Attention(dropout = dropout)
        attention2 = Attention(dropout = dropout)
    # elif attention == 'MultiHead':
    #     attention1 = MultiHeadAttention(num_heads=4)
    #     attention2 = MultiHeadAttention(num_heads=4)

    
    y = Conv2D(64, (5, 1), activation='relu', data_format=format)(multi_input)
    
    y = Conv2D(64, (5, 1), activation='relu', data_format=format)(y)  

    y = Conv2D(64, (5, 1), activation='relu', data_format=format)(y)

    y = Conv2D(64, (5, 1), activation='relu', data_format=format)(y)

    y = Permute((2, 1, 3))(y)
  
    y = Reshape((int(y.shape[1]), int(y.shape[2]) * int(y.shape[3])))(y)
   
    #adding attention layer, type of attention specified above

    query_encoding1 = Dense(300, activation='relu')(y)

    value_encoding1 = Dense(300, activation='relu')(y)

    y = attention1([query_encoding1,value_encoding1]) #attention layer - more like self attention but used along with conv and lstm
    
    y = LSTM(128,dropout=0.25,return_sequences=True)(y)

    query_encoding2 = Dense(int(y.shape[2]), activation='relu')(y)

    value_encoding2 = Dense(int(y.shape[2]), activation='relu')(y)

    y = attention2([query_encoding2,value_encoding2])

    y = LSTM(128)(y)

    # y = attention([y,y])

    return Model(inputs=multi_input, outputs=y) 

In [ ]:
# from tensorflow.python.framework.ops import disable_eager_execution
# disable_eager_execution()

In [ ]:
# =============================================================================
# Construct the training process
# The Si layers are from 5 to 21
# =============================================================================
# from keras.models import model_from_json
# json_file = open('/content/drive/MyDrive/MITACS_Nithin/juravinski/T-MTL_Models/trimtl_model_usall.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# model = model_from_json(loaded_model_json)
# # load weights into new model
# model.load_weights("/content/drive/MyDrive/MITACS_Nithin/juravinski/T-MTL_Models/trimtl_model_usall.h5")
# print("Loaded model from disk")
# print(model.summary())
# adam_optim = Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999)
# model.compile(loss=multi_task_loss(model,ALPHA), optimizer=adam_optim)

count = 0
epochs = 2

##Alternate TF Model Loading

In [ ]:
#get the triplets for training process
def get_triplets(xtrain):
    Anchor = xtrain[:,0,:].reshape(-1,1,SLIDING_WINDOW_LENGTH,NUM_CHANNEL)
    Positive = xtrain[:,1,:].reshape(-1,1,SLIDING_WINDOW_LENGTH,NUM_CHANNEL)
    Negative = xtrain[:,2,:].reshape(-1,1,SLIDING_WINDOW_LENGTH,NUM_CHANNEL)
    return Anchor,Positive,Negative


In [ ]:
# generate triplets and store them in lists: X_train_,y_train_; X_val and y_val just from one subject, and they're arrays
# divide into anchor,positive,and negative as input
def get_train_val(x_train_flat,y_train):
    print('start generating triplets')
    m = 50
    n = 20
    X_train_ = []
    y_train_ = []
    X_val_=[]
    y_val_=[]
    anchor_,positive_,negative_, anchor_test, positive_test, negative_test = [],[],[],[],[],[]
    for i in range(10):
        tempx,tempy = generate_triplet(x_train_flat[i],y_train[i].ravel(),ap_pairs=m,an_pairs=m)
        X_train_.append(tempx)
        y_train_.append(tempy)
        tempa,tempp,tempn = get_triplets(tempx)
        anchor_.append(tempa)
        positive_.append(tempp)
        negative_.append(tempn)
        
        tempx,tempy = generate_triplet(x_val_flat[i],y_val[i].ravel(),ap_pairs=n,an_pairs=n)
        X_val_.append(tempx)
        y_val_.append(tempy)
        tempa,tempp,tempn = get_triplets(tempx)
        anchor_test.append(tempa)
        positive_test.append(tempp)
        negative_test.append(tempn)
        
    return anchor_,positive_,negative_,anchor_test, positive_test,negative_test, y_train_, y_val_

In [ ]:
def make_ytrain(y):
    y_onehot = lb.transform(y)
    y_onehot = np.float32(y_onehot)
    res = []
    for i in range(10):
        res.append(y_onehot)   
    return res

def convert_list(my_list):
    length = len(my_list)
    if length==0:
        return
    res = my_list[0]
    for i in range(1,length):
        res = np.vstack((res,my_list[i]))
    return res

def convert_y(my_list):
    length = len(my_list)
    if length==0:
        return
    res = my_list[0]
    for i in range(1,length):
        res = np.concatenate((res,my_list[i]))
    return res

In [ ]:
anchor_input = Input((1,SLIDING_WINDOW_LENGTH,NUM_CHANNEL), name='anchor_input')
positive_input = Input((1,SLIDING_WINDOW_LENGTH,NUM_CHANNEL), name='positive_input')
negative_input = Input((1,SLIDING_WINDOW_LENGTH,NUM_CHANNEL), name='negative_input')

# Shared embedding layer for positive and negative items
Shared_DNN = Conv_Att_LSTM(attention='Luong')

encoded_anchor = Shared_DNN(anchor_input)
encoded_positive = Shared_DNN(positive_input)
encoded_negative = Shared_DNN(negative_input)

merged_vector = concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1, name='merged_layer')

finalAct = 'softmax'
sub1 = Dense(NUM_CLASSES,use_bias=True,activation=finalAct)(merged_vector)
sub2 = Dense(NUM_CLASSES,use_bias=True,activation=finalAct)(merged_vector)
sub3 = Dense(NUM_CLASSES,use_bias=True,activation=finalAct)(merged_vector)
sub4 = Dense(NUM_CLASSES,use_bias=True,activation=finalAct)(merged_vector) 

sub5 = Dense(NUM_CLASSES,use_bias=True,activation=finalAct)(merged_vector)
sub6 = Dense(NUM_CLASSES,use_bias=True,activation=finalAct)(merged_vector)
sub7 = Dense(NUM_CLASSES,use_bias=True,activation=finalAct)(merged_vector)
sub8 = Dense(NUM_CLASSES,use_bias=True,activation=finalAct)(merged_vector) 

sub9 = Dense(NUM_CLASSES,use_bias=True,activation=finalAct)(merged_vector)
sub10 = Dense(NUM_CLASSES,use_bias=True,activation=finalAct)(merged_vector)
# sub11 = Dense(NUM_CLASSES,use_bias=True,activation=finalAct)(merged_vector)
# sub12 = Dense(NUM_CLASSES,use_bias=True,activation=finalAct)(merged_vector) 

# sub13 = Dense(NUM_CLASSES,use_bias=True,activation=finalAct)(merged_vector)  
# sub14 = Dense(NUM_CLASSES,use_bias=True,activation=finalAct)(merged_vector)  

model = Model(inputs=[anchor_input,positive_input, negative_input], 
            outputs=[sub1,sub2,sub3,sub4,sub5,sub6,sub7,sub8,sub9,sub10])
model.load_weights("/content/drive/MyDrive/MITACS_Nithin/juravinski/T-MTL_Models/trimtl_model_usall_new_rest.h5")
# print("Loaded model from disk")
# print(model.summary())
adam_optim = Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999)
model.compile(loss=multi_task_loss(model,ALPHA), optimizer=adam_optim)
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 1, 60, 9)     0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 1, 60, 9)     0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 1, 60, 9)     0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 128)          554560      anchor_input[0][0]               
           

In [ ]:
class MyCustomCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    gc.collect()

In [ ]:

for count in tqdm(range(epochs)):

#Get the training and validation data for current epoch
    anchor_,positive_,negative_,anchor_test,positive_test,negative_test,y_train_,y_val_ = get_train_val(x_train_flat,y_train)

    #freeze all Si layers(from 5 to 20)
    for i in range(5,15):
        model.layers[i].trainable = False
        
    #fix all Si and update L first
    #if count>0:
    trainset_anchor = convert_list(anchor_)
    trainset_Positive = convert_list(positive_)
    trainset_Negative = convert_list(negative_)
    validation_anchor = convert_list(anchor_test)
    validation_positive = convert_list(positive_test)
    validation_negative = convert_list(negative_test)
    traintarget = convert_y(y_train_)
    target_onehot = make_ytrain(traintarget)
    testtarget = convert_y(y_val_)
    test_onehot = make_ytrain(testtarget)
    print(trainset_anchor.shape,traintarget.shape)
    #make L layers trainable   
    for i in range(11):
        model.layers[3].layers[i].trainable = True
    #model.compile(loss=custom_loss(model),

    model.compile(loss=multi_task_loss(model,ALPHA),optimizer = adam_optim)
        
    model.fit([trainset_anchor,trainset_Positive,trainset_Negative],
    y=target_onehot,validation_data=([validation_anchor,validation_positive,validation_negative],test_onehot), batch_size=512, epochs=1)

#Before training Si, freeze L
    for i in range(11):
        model.layers[3].layers[i].trainable = False

    #train Si(i=5 to 20) branch, freeze all the other S branches
    for j in range(5,15):
        #freeze all Si
        for i in range(5,15):
            model.layers[i].trainable = False
        #make current Si trainable
        model.layers[j].trainable = True
        trainset_anchor = anchor_[j-5]
        trainset_Positive = positive_[j-5]
        trainset_Negative = negative_[j-5]
        y=make_ytrain(y_train_[j-5])
        
        validation_anchor = anchor_test[j-5]
        validation_Positive = positive_test[j-5]
        validation_Negative = negative_test[j-5]
        test_ = make_ytrain(y_val_[j-5])
    
        model.compile(loss=multi_task_loss(model,ALPHA),metrics = ['accuracy'], optimizer=adam_optim)
            # is that necessary to do cross validation for each branch?
        
        model.fit([trainset_anchor,trainset_Positive,trainset_Negative],y,
            validation_data=([validation_anchor,validation_Positive,validation_Negative],test_),
            batch_size=512, 
            epochs=1)
            
        #save model to file every epochs
#       trained_model = Model(inputs=anchor_input, outputs=[sub1,sub2,sub3,sub4,sub5,sub6,sub7,sub8,sub9,
#                                sub10,sub11,sub12,sub13,sub14,sub15,sub16])
    # model.save('/content/drive/MyDrive/MITACS_Nithin/juravinski/T-MTL_Models/trimtl_model_usall_tf',save_format='tf')


    model_json = model.to_json()
    with open("/content/drive/MyDrive/MITACS_Nithin/juravinski/T-MTL_Models/trimtl_model_usall_new_rest.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("/content/drive/MyDrive/MITACS_Nithin/juravinski/T-MTL_Models/trimtl_model_usall_new_rest.h5") 

    gc.collect() 

    del anchor_
    del positive_
    del negative_
    del anchor_test
    del positive_test
    del negative_test
    del y_train_
    del y_val_

    anchor_,positive_,negative_,anchor_test,positive_test,negative_test,y_train_,y_val_ =[],[],[],[],[],[],[],[]


  0%|          | 0/2 [00:00<?, ?it/s]

start generating triplets
(100000, 1, 60, 9) (100000,)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 100000 samples, validate on 16000 samples
Epoch 1/1
100000/100000 [==============================] - 64s 645us/step - loss: 3221.2813 - dense_1_loss: 322.1283 - dense_2_loss: 322.1278 - dense_3_loss: 322.1280 - dense_4_loss: 322.1279 - dense_5_loss: 322.1279 - dense_6_loss: 322.1278 - dense_7_loss: 322.1280 - dense_8_loss: 322.1281 - dense_9_loss: 322.1280 - dense_10_loss: 322.1288 - val_loss: 3221.4038 - val_dense_1_loss: 322.1382 - val_dense_2_loss: 322.1402 - val_dense_3_loss: 322.1389 - val_dense_4_loss: 322.1400 - val_dense_5_loss: 322.1396 - val_dense_6_loss: 322.1393 - val_dense_7_loss: 322.1418 - val_dense_8_loss: 322.1405 - val_dense_9_loss: 322.1398 - val_dense_10_loss: 322.1388
Train on 10000 samples, validate on 1600 samples
Epoch 1/1
10000/10000 [==============================] - 5s 453us/step - loss: 3221.1781 - den

 50%|█████     | 1/2 [05:44<05:44, 344.25s/it]

start generating triplets
(100000, 1, 60, 9) (100000,)
Train on 100000 samples, validate on 16000 samples
Epoch 1/1
100000/100000 [==============================] - 65s 650us/step - loss: 3224.5075 - dense_1_loss: 322.4509 - dense_2_loss: 322.4507 - dense_3_loss: 322.4507 - dense_4_loss: 322.4506 - dense_5_loss: 322.4508 - dense_6_loss: 322.4504 - dense_7_loss: 322.4503 - dense_8_loss: 322.4507 - dense_9_loss: 322.4505 - dense_10_loss: 322.4510 - val_loss: 3224.6952 - val_dense_1_loss: 322.4684 - val_dense_2_loss: 322.4691 - val_dense_3_loss: 322.4683 - val_dense_4_loss: 322.4680 - val_dense_5_loss: 322.4696 - val_dense_6_loss: 322.4692 - val_dense_7_loss: 322.4685 - val_dense_8_loss: 322.4686 - val_dense_9_loss: 322.4687 - val_dense_10_loss: 322.4692
Train on 10000 samples, validate on 1600 samples
Epoch 1/1
10000/10000 [==============================] - 5s 543us/step - loss: 3224.4127 - dense_1_loss: 322.4411 - dense_2_loss: 322.4411 - dense_3_loss: 322.4413 - dense_4_loss: 322.4414 

100%|██████████| 2/2 [17:41<00:00, 530.80s/it]


In [ ]:
anchor_,positive_,negative_,anchor_test, positive_test, negative_test,y_train_,y_val_ = [],[],[],[],[],[],[],[]

In [ ]:
# for i in range(11):
#     model.layers[3].layers[i].trainable = False

In [ ]:
anchor_,positive_,negative_,y_train_ =[],[],[],[]

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("trimtl_modelusfinalall.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("trimtl_weightsusfinalall.h5")
print("Saved model to disk")